In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fpdf import FPDF
import shutil
from IPython.display import display

# ==============================================================================
# 1. CONFIGURAÇÃO INICIAL
# ==============================================================================

# --- Função de Limpeza ---
def limpar_e_converter_colunas_numericas(df):
    """
    Itera sobre as colunas de um DataFrame, substitui vírgulas por pontos
    em colunas de texto e tenta convertê-las para um formato numérico.
    """
    print("Iniciando limpeza de colunas numéricas...")
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_numeric(df[col].str.replace(',', '.', regex=False))
                print(f"-> Coluna '{col}' convertida para numérico.")
            except (AttributeError, ValueError):
                # Se a conversão falhar, a coluna é mantida como texto.
                pass
    print("Limpeza concluída.")
    return df

# --- Definir caminhos ---
data_dir = '../../dados_brutos/datafolha_2928278/'
output_dir = '../../analises/'
temp_img_dir = os.path.join(output_dir, 'temp_images/')

# --- Criar diretórios se não existirem ---
os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_img_dir, exist_ok=True)

try:
    # --- Caminho completo para o arquivo ---
    arquivo_path = os.path.join(data_dir, 'PM3283_BASE_TIPO1.xlsx')
    
    # --- Passo 1: Verificar as planilhas disponíveis no arquivo ---
    print(f"Analisando o arquivo: '{os.path.basename(arquivo_path)}'")
    excel_file = pd.ExcelFile(arquivo_path)
    sheet_names = excel_file.sheet_names
    print(f"Planilhas encontradas: {sheet_names}")

    # --- Passo 2: Carregar a planilha desejada ---
    # O código tentará carregar a planilha que você especificou.
    # Se não encontrar, carregará a primeira da lista como alternativa.
    sheet_para_carregar = 'PM3283_BASE_TIPO1'
    if sheet_para_carregar not in sheet_names:
        print(f"AVISO: A planilha '{sheet_para_carregar}' não foi encontrada.")
        sheet_para_carregar = sheet_names[0] # Usa a primeira planilha disponível
        print(f"Carregando a primeira planilha como alternativa: '{sheet_para_carregar}'")
        
    df1 = pd.read_excel(excel_file, sheet_name=sheet_para_carregar)
    print(f"DataFrame 'df1' carregado da planilha '{sheet_para_carregar}'.")

    # --- Passo 3: Limpar os dados carregados ---
    # Aplica a função para corrigir os erros de formato de número (com vírgula)
    df1 = limpar_e_converter_colunas_numericas(df1)

    # --- Passo 4: Verificar o resultado ---
    print("\nVerificação do DataFrame final:")
    df1.info()
    print("\nPrimeiras 5 linhas do df1:")
    print(df1.head())

except FileNotFoundError:
    print(f"\nERRO: Arquivo não encontrado. Verifique se o caminho está correto: '{arquivo_path}'")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")
#df2 = pd.read_csv(os.path.join(data_dir, 'PM3283_BASE_TIPO2.sav'))

Analisando o arquivo: 'PM3283_BASE_TIPO1.xlsx'
Planilhas encontradas: ['Sheet1', 'Sheet2', 'Sheet3']
AVISO: A planilha 'PM3283_BASE_TIPO1' não foi encontrada.
Carregando a primeira planilha como alternativa: 'Sheet1'
DataFrame 'df1' carregado da planilha 'Sheet1'.
Iniciando limpeza de colunas numéricas...
Limpeza concluída.

Verificação do DataFrame final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66500 entries, 0 to 66499
Columns: 1399 entries, D_R to N168E
dtypes: float64(516), int64(603), object(280)
memory usage: 709.8+ MB

Primeiras 5 linhas do df1:
   D_R    dig  nquest    cpd  telefone  inloco  cidade  uf  reg_vit  porte  \
0  NaN  11665       1  59521         2       2     100  SP        1      1   
1  NaN  11665       2  59522         2       2     100  SP        1      1   
2  NaN  11665       4  59986         1       2     100  SP        1      1   
3  NaN  11665       5  59682         1       2     100  SP        1      1   
4  NaN  11665       6  59695         1   

In [20]:
# --- Renomear colunas do DataFrame df1 ---
# Dicionário unificado com o mapeamento completo de colunas para perguntas
mapa_de_colunas_unificado = {
    # Bloco de Identificação e Dados Sociodemográficos
    'p1': 'quantas pessoas com 16 anos ou mais moram na sua casa?',
    'p1_mulheres': 'e quantas são mulheres com 16 anos ou mais?',
    'p2': 'o(a) sr(a) poderia me dizer o primeiro nome e a idade de cada pessoa com 16 anos ou mais, incluindo o(a) sr(a), e começando do mais velho para o mais novo?',
    'p2a': 'quantas pessoas com menos de 16 anos moram em sua casa?',
    'p2b': 'por favor, me diga o nome e a idade das pessoas da sua casa com menos de 16 anos',
    'p3': 'o(a) sr(a) sempre morou nesta cidade?',
    'p4': 'há quanto tempo aproximadamente o(a) sr(a) mora nessa cidade?',
    'p5': 'atualmente o(a) sr(a) é: solteiro(a); casado(a); amigado, amasiado (união consensual); separado(a) judicialmente / divorciado(a); separado (a) não oficialmente ou viúvo(a)?',
    'p6': 'qual das seguintes categorias descreve melhor sua cor: branca; preta; parda; amarela ou indígena?',
    'p7': 'qual é o curso mais elevado que já concluiu? até que série o(a) sr(a) estudou?',
    'p8': 'em relação à orientação sexual, o(a) sr(a) diria que é : heterossexual, homossexual, ou bissexual?',
    'p9': 'atualmente, o(a) sr(a) trabalha? qual é a sua ocupação principal?',
    'p10': 'o seu trabalho fica na região central da cidade?',
    'p11': 'qual é aproximadamente a renda total mensal de todas as pessoas que moram neste domicílio, somando todas as fontes como salário, pensão, aposentadoria, benefícios sociais, aluguéis, bicos?',
    'p12': 'vou ler uma lista de religiões para que o(a) sr(a) me indique se alguma delas é a sua',

    # Bloco de Atividades Cotidianas
    'p13a': 'durante a semana, de segunda a sexta-feira, onde o(a) sr(a) costuma ficar a maior parte da manhã, das 06h ao meio-dia em casa ou fora de casa?',
    'p13b': 'durante a semana, de segunda a sexta-feira, onde o(a) sr(a) costuma ficar a maior parte da tarde, do meio-dia às 18h em casa ou fora de casa?',
    'p13c': 'durante a semana, de segunda a sexta-feira, onde o(a) sr(a) costuma ficar a maior parte da noite, das 18h01 à meia-noite em casa ou fora de casa?',
    'p13d': 'durante a semana, de segunda a sexta-feira, onde o(a) sr(a) costuma ficar a maior parte da madrugada, da meia-noite às 06h em casa ou fora de casa?',
    'p14a': 'nos últimos 30 dias o(a) sr(a) saiu para ir ao cinema?',
    'p14b': 'nos últimos 30 dias o(a) sr(a) saiu para ir à praia ou parque?',
    'p14c': 'nos últimos 30 dias o(a) sr(a) saiu para comer fora de casa, sem contar os dias de trabalho?',
    'p14d': 'nos últimos 30 dias o(a) sr(a) saiu para visitar amigos ou parentes?',
    'p14e': 'nos últimos 30 dias o(a) sr(a) saiu para assistir algum evento esportivo amador ou profissional ao vivo?',
    'p14f': 'nos últimos 30 dias o(a) sr(a) saiu para praticar alguma atividade esportiva ou física como futebol, basquete, tênis, boliche, natação, aula de dança, ginástica ou fazer caminhada?',
    'p14g': 'nos últimos 30 dias o(a) sr(a) saiu para ir a missa, culto, atividade religiosa ou mística?',
    'p14h': 'nos últimos 30 dias o(a) sr(a) saiu para ir ao shopping ou centro comercial?',
    'p14i': 'nos últimos 30 dias o(a) sr(a) saiu para feiras populares?',
    'p14j': 'nos últimos 30 dias o(a) sr(a) saiu para assistir a algum show ou espetáculo de música, dança?',
    'p14k': 'nos últimos 30 dias o(a) sr(a) saiu para ir a algum bar ou casa noturna?',
    'p15': 'quais os três principais meios de transporte que o(a) sr(a) mais costuma utilizar durante a semana?',
    'p16': 'quem (é) são os responsáveis por cuidar e supervisionar as crianças ou adolescentes desta residência, quando não estiverem na escola ou creche?',

    # Bloco de Mapeamento de Crimes (Perguntas Gerais)
    'p17': 'o(a) sr(a) ou alguém da sua casa possui ou possuiu algum carro, caminhão ou caminhonete para uso próprio nos últimos 12 meses?',
    'p18': 'o(a) sr(a) ou alguém da sua casa possui ou possuiu alguma moto, motocicleta ou lambreta para uso próprio nos últimos 12 meses?',
    'p19': 'alguma vez, alguém furtou o(s) seu(s) carro(s), caminhão(ões) ou caminhonete(s) ou de alguém de sua casa, isto é, levou sem utilizar força ou fazer ameaça?',
    'p19_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p20': 'alguma vez, alguém roubou o seu(s) carro(s), caminhão(ões) ou caminhonete(s) ou de alguém de sua casa com o uso de violência ou ameaça?',
    'p20_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p21': 'alguma vez, alguém furtou a(s) sua(s) moto(s), motocicleta(s) ou lambreta(s) ou de alguém de sua casa, isto é, levou sem utilizar força ou fazer ameaça?',
    'p21_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p22': 'alguma vez, alguém roubou a(s) sua(s) moto(s), motocicleta(s) ou lambreta(s) ou de alguém de sua casa com o uso de violência ou ameaça?',
    'p22_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p23': 'alguma vez, o(a) sr(a) teve qualquer evidência de que alguém furtou qualquer um outro(s) bem(ns) do(a) sr(a), isto é, levou sem utilizar força ou fazer ameaça?',
    'p23_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p24': 'alguma vez, alguém roubou algum objeto seu, com o uso de violência ou ameaça?',
    'p24_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p25': 'alguma vez o(a) sr(a) foi vítima de sequestro?',
    'p25_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    'p26': 'e alguma vez o(a) sr(a) foi vítima de sequestro relâmpago?',
    'p26_ultimos12meses': 'se sim, isso ocorreu nos últimos doze meses?',
    
    # Fraude e Estelionato
    'p27a': 'nos últimos 12 meses, o(a) sr(a) sofreu alguma fraude contra o seu cartão de crédito?',
    'p27b': 'nos últimos 12 meses, o(a) sr(a) sofreu fraude com cheque?',
    'p27c': 'nos últimos 12 meses, o(a) sr(a) recebeu notas de dinheiro falso?',
    'p27d': 'nos últimos 12 meses, o(a) sr(a) pagou por algum produto que não foi entregue?',
    'p27e': 'nos últimos 12 meses, o(a) sr(a) foi vítima de fraude em algum investimento que realizou?',
    'p27f': 'nos últimos 12 meses, o(a) sr(a) teve o seu celular clonado, ou seja, o seu número foi usado indevidamente por outra pessoa?',
    'p27g': 'nos últimos 12 meses, o(a) sr(a) teve linha telefônica residencial violada ou desviada?',
    'p27h': 'nos últimos 12 meses, o(a) sr(a) sofreu fraude de documentos pessoais?',
    'p27i': 'nos últimos 12 meses, o(a) sr(a) foi vítima de algum outro tipo de fraude?',
    'p27j': 'nos últimos 12 meses, o(a) sr(a) sofreu fraudes pela internet (sites de bancos falsos, etc)?',
    'p27a_ultima': 'qual dessas foi a última?',

    # Acidente
    'p28a': 'nos últimos 12 meses, o (a) sr (a) foi vítima de afogamento?',
    'p28b': 'nos últimos 12 meses, o (a) sr (a) foi vítima de queda?',
    'p28c': 'nos últimos 12 meses, o (a) sr (a) foi vítima de asfixia?',
    'p28d': 'nos últimos 12 meses, o (a) sr (a) foi vítima de eletrocução ou seja, choque de alta voltagem?',
    'p28e': 'nos últimos 12 meses, o (a) sr (a) foi vítima de acidente de trânsito? (inclui atropelamento)',
    'p28f': 'nos últimos 12 meses, o (a) sr (a) foi vítima de outro tipo de acidente?',
    'p29a': 'nos últimos 12 meses, algum conhecido seu foi vítima de afogamento?',
    'p29b': 'nos últimos 12 meses, algum conhecido seu foi vítima de queda?',
    'p29c': 'nos últimos 12 meses, algum conhecido seu foi vítima de asfixia?',
    'p29d': 'nos últimos 12 meses, algum conhecido seu foi vítima de eletrocução, ou seja, choque de alta voltagem?',
    'p29e': 'nos últimos 12 meses, algum conhecido seu foi vítima de acidente de trânsito? (inclui atropelamento)',
    'p29f': 'nos últimos 12 meses, algum conhecido seu foi vítima de outro tipo de acidente?',
    
    # Agressões ou Ameaças
    'p30a': 'nos últimos 12 meses, o(a) sr(a) foi vítima de insulto, humilhação ou xingamento (ofensa verbal)?',
    'p30a_quantas': 'quantas vezes?',
    'p30b': 'nos últimos 12 meses, o(a) sr(a) foi vítima de ameaça de apanhar, empurrar ou chutar?',
    'p30b_quantas': 'quantas vezes?',
    'p30c': 'nos últimos 12 meses, o(a) sr(a) foi vítima de ameaça com faca ou arma de fogo?',
    'p30c_quantas': 'quantas vezes?',
    'p30d': 'nos últimos 12 meses, o(a) sr(a) foi vítima de amedrontamento ou perseguição?',
    'p30d_quantas': 'quantas vezes?',
    'p30e': 'nos últimos 12 meses, o(a) sr(a) foi vítima de batida, empurrão ou chute?',
    'p30e_quantas': 'quantas vezes?',
    'p30f': 'nos últimos 12 meses, o(a) sr(a) foi vítima de lesão provocada por algum objeto que lhe foi atirado?',
    'p30f_quantas': 'quantas vezes?',
    'p30g': 'nos últimos 12 meses, o(a) sr(a) foi vítima de espancamento ou tentativa de estrangulamento?',
    'p30g_quantas': 'quantas vezes?',
    'p30h': 'nos últimos 12 meses, o(a) sr(a) foi vítima de esfaqueamento ou tiro?',
    'p30h_quantas': 'quantas vezes?',
    'p30i': 'nos últimos 12 meses, o(a) sr(a) foi vítima de ameaça de ter seus bens e documentos subtraídos/tomados/retirados/ por parentes, companheiros ou conhecidos?',
    'p30i_quantas': 'quantas vezes?',
    'p30j': 'nos últimos 12 meses, o(a) sr(a) foi vítima de alguma outra ameaça ou agressão?',
    'p30j_quantas': 'quantas vezes?',
    'p30b_ultima': 'qual dessas foi a última agressão ou ameaça sofrida?',

    # Ofensa Sexual e Discriminação
    'p31': 'alguma vez, alguém fez ou tentou fazer isto com o(a) sr(a)?',
    'p31a': 'isso aconteceu nos últimos 12 meses?',
    'p32a': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por sua cor ou raça?',
    'p32a_quantas': 'quantas vezes?',
    'p32b': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por ser homem / mulher?',
    'p32b_quantas': 'quantas vezes?',
    'p32c': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por sua orientação ou opção sexual?',
    'p32c_quantas': 'quantas vezes?',
    'p32d': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por doença?',
    'p32d_quantas': 'quantas vezes?',
    'p32e': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por deficiência física?',
    'p32e_quantas': 'quantas vezes?',
    'p32f': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por sua religião?',
    'p32f_quantas': 'quantas vezes?',
    'p32g': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação pelo lugar onde mora?',
    'p32g_quantas': 'quantas vezes?',
    'p32h': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação por sua classe social?',
    'p32h_quantas': 'quantas vezes?',
    'p32i': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação pela sua escolaridade?',
    'p32i_quantas': 'quantas vezes?',
    'p32j': 'nestes últimos 12 meses, o(a) sr(a) sofreu algum tipo de discriminação pela sua preferência política?',
    'p32j_quantas': 'quantas vezes?',
    'p32b_ultima': 'qual dessas foi a última?',
    
    # Detalhamento de Crimes
    'p33': 'quantas vezes o(a) sr(a) ou alguém da sua casa teve o carro, caminhão ou caminhonete furtado nos últimos 12 meses?',
    'p38': 'quantas vezes o(a) sr(a) ou alguém da sua casa teve o carro, caminhão ou caminhonete roubado nos últimos 12 meses?',
    'p48': 'quantas vezes o(a) sr(a) ou alguém da sua casa teve a moto, motocicleta ou lambreta furtada nos últimos 12 meses?',
    'p53': 'quantas vezes o(a) sr(a) ou alguém da sua casa teve a moto, motocicleta ou lambreta roubada nos últimos 12 meses?',
    'p63': 'quantas vezes seus objetos foram furtados nos últimos 12 meses?',
    'p69': 'quantas vezes seus objetos foram roubados nos últimos 12 meses?',
    'p80': 'quantas vezes o(a) sr(a) foi vítima de (sequestro ou sequestro relâmpago) nos últimos 12 meses?',
    'p91': 'quantas vezes (leia a ocorrência da p.27a) ocorreu nos últimos 12 meses?',
    'p118': 'quantas vezes o(a) sr(a) foi vítima de ofensa sexual nos últimos 12 meses?',

    # Vizinhança e Percepção de Segurança
    'p139': 'há quanto tempo o(a) sr(a) mora na sua vizinhança?',
    'p140': 'se pudesse escolher, o(a) sr(a) gostaria de continuar morando na sua vizinhança; mudaria para outro lugar do mesmo bairro; mudaria para outro bairro da cidade; mudaria para outra cidade do estado; mudaria para outro estado do brasil ou mudaria para outro país?',
    'p141': 'em relação aos seus vizinhos, o(a) sr(a) diria que: conhece todos ou quase todos; conhece muitos; conhece alguns ou não conhece nenhum?',
    'p142': 'com relação aos seus vizinhos, o(a) sr(a) diria que: confia na maioria deles; confia em alguns deles ou não confia em nenhum deles?',
    'p143a': 'em quais das seguintes situações o(a) sr(a) acha que poderia contar ou não com seus vizinhos se precisasse: para auxiliar em caso de doença ou problema de saúde?',
    'p143b': 'em quais das seguintes situações o(a) sr(a) acha que poderia contar ou não com seus vizinhos se precisasse: para pedir dinheiro emprestado?',
    'p144a': 'algum vizinho costuma pedir ajuda ou favores ao(a) sr(a) como auxiliar em caso de doença ou problema de saúde frequentemente, raramente ou nunca?',
    'p144b': 'algum vizinho costuma pedir ajuda ou favores ao(a) sr(a) como pedir dinheiro emprestado frequentemente, raramente ou nunca?',
    'p145a': 'pensando na sua vizinhança, o(a) sr(a) diria que pode contar com seus vizinhos para tomar alguma providência se: observar jovens matando aula e ficando a toa na rua',
    'p145b': 'pensando na sua vizinhança, o(a) sr(a) diria que pode contar com seus vizinhos para tomar alguma providência se: observar jovens depredando prédios',
    'p146a': 'vou ler algumas frases e gostaria que o(a) sr(a) dissesse se concorda ou se discorda com cada uma delas, pensando na sua vizinhança: as pessoas daqui são dispostas ajudar seus vizinhos',
    'p146b': 'vou ler algumas frases e gostaria que o(a) sr(a) dissesse se concorda ou se discorda com cada uma delas, pensando na sua vizinhança: as pessoas nesta vizinhança são muito unidas',
    'p147a': 'na sua vizinhança existem ou não prédios, casas ou galpões abandonados?',
    'p147b': 'na sua vizinhança existem ou não carros abandonados, arrebentados ou desmontados nas ruas?',
    'p152': 'como o(a) sr(a) se sente ao andar nas ruas do bairro onde reside durante o dia',
    'p153': 'como o(a) sr(a) se sente ao andar nas ruas do bairro onde reside durante a noite',
    'p162a': 'por causa da violência, o(a) sr(a): evita sair à noite ou chegar muito tarde em casa?',
    'p162b': 'por causa da violência, o(a) sr(a): muda de caminho entre a casa e o trabalho ou a escola ou lazer?',
    'p163a': 'pensando na sua vizinhança, o(a) sr.(a) teme ser assaltado(a)?',
    'p163b': 'pensando na sua vizinhança, o(a) sr.(a) teme ter o carro ou moto roubado num assalto?',
    'p164': 'o(a) sr(a) diria que, nos últimos 12 meses, a criminalidade na sua cidade aumentou, diminuiu ou ficou igual?',
    'p165': 'o(a) sr(a) diria que, nos últimos 12 meses, a criminalidade na sua vizinhança aumentou, diminuiu ou ficou igual?',
    'p166a': 'o(a) sr(a) tem medo de: ter sua residência invadida ou arrombada?',
    'p167a': 'o(a) sr(a) acredita que pode se tornar vítima de algum desses crimes no próximo ano? acredita que pode: ter sua residência invadida ou arrombada?',
    
    # Polícia e Finalização
    'p170': 'de uma maneira geral, em se tratando da polícia militar o(a) sr(a): confia muito; confia um pouco ou não confia?',
    'p171a': 'agora vou citar algumas frases e para cada frase eu gostaria que o(a) sr(a) me dissesse se concorda ou discorda?: os policiais militares sabem como agir em situações de risco e perigo',
    'p172a': 'como o(a) sr(a) avalia o atendimento da polícia militar em relação a(ao): punição dos policiais com mau comportamento?',
    'p176': 'de uma maneira geral, em se tratando da polícia civil o(a) sr(a): confia muito; confia um pouco ou não confia?',
    'p177a': 'como o(a) sr(a) avalia o atendimento da polícia civil em relação a(ao): investigação de crimes?',
    'p180': 'para finalizar, o(a) sr(a). ou alguém da sua casa possui uma arma de fogo?',
    'p181a': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem televisão colorida em sua casa? (se sim) quantos?',
    'p181b': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem automóvel de passeio em sua casa? (se sim) quantos?',
    'p181c': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem rádio sem contar o do carro em sua casa? (se sim) quantos?',
    'p181d': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem empregada doméstica mensalista em sua casa? (se sim) quantos?',
    'p181e': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem máquina de lavar roupas em sua casa? (se sim) quantos?',
    'p181f': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem videocassete ou dvd em sua casa? (se sim) quantos?',
    'p181g': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem banheiro em sua casa? (se sim) quantos?',
    'p181h': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem geladeira em sua casa? (se sim) quantos?',
    'p181i': 'apenas para classificação, o(a) sr(a) poderia me dizer se tem freezer (aparelho independente ou parte da geladeira duplex) em sua casa? (se sim) quantos?',
    'p182': 'até que ano da escola o chefe da família estudou?',
    'p183': 'tipo de moradia'
}

df1 = df1.rename(columns= mapa_de_colunas_unificado)
df2 = df2.rename(columns= mapa_de_colunas_unificado)

display(df1.head())
display(df2.head())


,PERFIL DA AMOSTRA,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51
0,(Resposta em %),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,TOTAL,REGIAO,NaN,NaN,NaN,NaN,NaN,NaN,...,ESCOLARIDADE,NaN,NaN,CRITÉRIO BRASIL,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,SUDESTE,NaN,NaN,NaN,NaN,SUL,NaN,...,Funda- mental,Médio,Supe- rior,A/B,A,B,C,D/E,D,E
3,NaN,NaN,NaN,TOTAL,ESTADO,NaN,NaN,NaN,TOTAL,ESTADO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,ES,MG,RJ,SP,NaN,PR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


""
